# 1. Data import

## 1.1 Importing the dataframes

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from statsmodels.stats.proportion import proportions_ztest
import math as math
from scipy import stats
import importlib
import project_functions

Here, we import functions we created in a previous notebook, so we can clean / reformat the column headers

In [2]:
importlib.reload(project_functions)

<module 'project_functions' from 'c:\\Users\\Carlos\\Desktop\\Vanguard-AB-Testing\\project_functions.py'>

In [3]:
# cleaning the columns : changing the columns headers to correct format
# for more information about this function --> cf project_functions.py notebook
clean_my_columns = project_functions.clean_my_columns

Now, we 'instance' our dataframes so we precise to Python to read them:

In [11]:
# Cargar el dataset demografic
df_pt1 = pd.read_csv('raw_data/df_final_web_data_pt_1.txt')
df_pt2 = pd.read_csv('raw_data/df_final_web_data_pt_2.txt')
df_users = pd.read_csv('raw_data/df_final_demo.txt')
df_exp =pd.read_csv('raw_data/df_final_experiment_clients.txt')

We apply column titles cleaning to every dataframe

In [ ]:
clean_my_columns(df_exp)

clean_my_columns(df_users)

clean_my_columns(df_pt1)

clean_my_columns(df_pt2)

## 1.2 Looking at the datraframes

### 1.2.0 Info about DF's

In [ ]:
df_users.info(include='all')

In [ ]:
df_exp.info(include='all')

In [ ]:
df_pt1.info(include='all')

In [ ]:
df_pt2.info(include='all')

### 1.2.1 DF "experiment participation"

The exp dataframe states whether a client (client_id) is being redirected on the original version (control) or to the new version (test)

In [ ]:
display(df_exp)
df_exp.shape

### 1.2.2 DF "Clients' Demographic"

In [ ]:
display(df_users)
df_users.shape

### 1.2.3 "Footprints Data" - 2 Tables

In [ ]:
display(df_pt1.head())
df_pt1.shape

In [ ]:
display(df_pt2.head())
df_pt2.shape

## 1.3 Combining tables

We're combining both group of tabless due to we have the same data columns within them.

### 1.3.1 Combining Footprints Tables:

In [15]:
# Combining df_pt_1 and df_pt_2:
df_footprint=pd.concat([df_pt1,df_pt2])

In [ ]:
#Now we take a overview about the concatenated table
display(df_footprint.head())

print(f"shape of pt1: {df_pt1.shape}")
print(f"shape of pt2: {df_pt2.shape}")
print(f"shape of the concat: {df_footprint.shape}")

Let's see how many unique clients we have data.

In [ ]:
# Number of clients
df_footprint['client_id'].nunique()

### 1.3.2 Combining Demo and Experiment tables

Experiment a demo tables have the exactly same shape and they have client_id in common, soy we are going to merge them to have the second column from experiment as another variable in our big table "demographics". 

In [19]:
df_clients = df_users.merge(df_exp, on='client_id', how='left')

In [ ]:
display(df_clients)
df_clients.shape

# 2. Cleaning Dataframes

## 2.1 Cleaning Clients Table

### 2.1.1 Duplicates

In [26]:
# Let's check how many duplicated rows there are.
duplicates_clients = df_clients.duplicated(keep=False)
df_duplicates_clients = df_clients[duplicates_clients]

print(df_duplicates_clients)

Empty DataFrame
Columns: [client_id, clnt_tenure_yr, clnt_tenure_mnth, clnt_age, gendr, num_accts, bal, calls_6_mnth, logons_6_mnth, Variation]
Index: []


In [ ]:
# Deleting all duplicates
df_clients_nd = df_clients.drop_duplicates(keep=False)

print(df_clients_nd)

### 2.1.2 Nulls

In [ ]:
missing_clients_data = df_clients_nd.isnull().sum()
print(missing_clients_data)

We procced to drop rows with nulls data (20109 rows)

In [ ]:
df_clients_nd_nn = df_clients_nd.dropna()

print(df_clients_nd_nn)

### 2.1.3 Column client_id

* Checking unique values

In [31]:
df_clients_nd_nn.shape

(50487, 10)

In [33]:
df_clients_nd_nn['client_id'].nunique()

50487

Unique values for id_client is the same than shape, so we assume it is cleaned.

### 2.1.4 Column clnt_tenure_yr

In [ ]:
display(df_clients_nd_nn['clnt_tenure_yr'])
print(f"Valor mínimo: {df_clients_nd_nn['clnt_tenure_yr'].min()}")
print(f"Valor mínimo: {df_clients_nd_nn['clnt_tenure_yr'].max()}")

We change the column datatype:

In [ ]:
df_clients_nd_nn['clnt_tenure_yr'] = df_clients_nd_nn['clnt_tenure_yr'].astype(int)

In [ ]:
df_clients_nd_nn['clnt_tenure_yr'].unique()

### 2.1.5 Column clnt_tenure_mnth

In [ ]:
display(df_clients_nd_nn['clnt_tenure_yr'])
print(f"Valor mínimo: {df_clients_nd_nn['clnt_tenure_yr'].min()}")
print(f"Valor mínimo: {df_clients_nd_nn['clnt_tenure_yr'].max()}")

We change the column datatype:

In [ ]:
df_clients_nd_nn['clnt_tenure_mnth'] = df_clients_nd_nn['clnt_tenure_mnth'].astype(int)

In [ ]:
df_clients_nd_nn['clnt_tenure_mnth'].unique()

Let's see if moonths and years of tenure are coherent. 

In [ ]:
# Comprobamos si hay filas donde los meses son mayores o iguales a 12 pero no coinciden con los años
df_inconsistencias = df_clients_nd_nn[(df_clients_nd_nn['clnt_tenure_mnth'] >= 12) & (df_clients_nd_nn['clnt_tenure_yr'] != df_clients_nd_nn['clnt_tenure_mnth'] // 12)]

# Mostrar filas con inconsistencias
print(df_inconsistencias)

Let's drop that inconsistences:

In [43]:
df_clients_nd_nn_in = df_clients_nd_nn.drop(df_inconsistencias.index)

### 2.1.6 Column client_age

In [ ]:
display(df_clients_nd_nn_in['clnt_age'])
print(f"Valor mínimo: {df_clients_nd_nn_in['clnt_age'].min()}")
print(f"Valor máximo: {df_clients_nd_nn_in['clnt_age'].max()}")

We round the client's age.We add the column.

In [44]:
df_clients_nd_nn_in['edad_redondeada'] = df_clients_nd_nn_in['clnt_age'].round()

Let's graph ages

In [ ]:
# Supongamos que tu DataFrame tiene una columna llamada 'edad'
df_clients_nd_nn_in['edad_redondeada'].value_counts().sort_index().plot(kind='bar', figsize=(10,6))

# Añadir título y etiquetas
plt.title('Distribución de Edades de los Clientes')
plt.xlabel('Edad')
plt.ylabel('Número de Clientes')

# Mostrar el gráfico
plt.show()

### 2.1.7 Column gender

In [45]:
display(df_clients_nd_nn_in['gendr'])
print(f"Valor mínimo: {df_clients_nd_nn_in['gendr'].min()}")
print(f"Valor máximo: {df_clients_nd_nn_in['gendr'].max()}")

Check unique values

In [ ]:
unique_genders = df_clients_nd_nn_in['gendr'].unique()
count_genders = df_clients_nd_nn_in['gendr'].value_counts()
print("\nValores únicos en la columna 'gendr':")
print(unique_genders)
print(count_genders)

Let's replace X by U in those 2 values with X gender. 

In [46]:
df_clients_nd_nn_in['gendr'] = df_clients_nd_nn_in['gendr'].replace('X', 'U')

In [47]:
# In case you wannna drop those.
# df_clients_nd_nn_in_gdr = df_clients_nd_nn_in[df_clients_nd_nn_in['gendr'] != 'X']

### 2.1.8 Column num_accts

In [ ]:
display(df_clients_nd_nn_in['num_accts'])
print(f"Valor mínimo: {df_clients_nd_nn_in['num_accts'].min()}")
print(f"Valor máximo: {df_clients_nd_nn_in['num_accts'].max()}")

We change the column datatype:

In [49]:
df_clients_nd_nn_in['num_accts'] = df_clients_nd_nn_in['num_accts'].astype(int)

In [ ]:
df_clients_nd_nn_in['num_accts'].value_counts()

### 2.1.9 Column bal

In [ ]:
display(df_clients_nd_nn_in['num_accts'])
print(f"Valor mínimo: {df_clients_nd_nn_in['num_accts'].min()}")
print(f"Valor máximo: {df_clients_nd_nn_in['num_accts'].max()}")

In [ ]:
df_clients_nd_nn_in['num_accts'].value_counts()

### 2.1.10 Column calls_6_month

In [ ]:
display(df_clients_nd_nn_in['calls_6_mnth'])
print(f"Valor mínimo: {df_clients_nd_nn_in['calls_6_mnth'].min()}")
print(f"Valor máximo: {df_clients_nd_nn_in['calls_6_mnth'].max()}")

In [ ]:
df_clients_nd_nn_in['calls_6_mnth'].value_counts()

Let's turn into int

In [54]:
df_clients_nd_nn_in['calls_6_mnth'] = df_clients_nd_nn_in['calls_6_mnth'].astype(int)

### 2.1.11 Column logons_6_mnth

In [ ]:
display(df_clients_nd_nn_in['logons_6_mnth'])
print(f"Valor mínimo: {df_clients_nd_nn_in['logons_6_mnth'].min()}")
print(f"Valor máximo: {df_clients_nd_nn_in['logons_6_mnth'].max()}")

In [ ]:
df_clients_nd_nn_in['logons_6_mnth'].value_counts()

In [57]:
df_clients_nd_nn_in['logons_6_mnth'] = df_clients_nd_nn_in['logons_6_mnth'].astype(int)

## 2.2 Cleaning Footprints Table

### 2.2.1 Duplicates

In [ ]:
# Let's check how many duplicated rows there are.
duplicates = df_footprint.duplicated(keep=False)
df_duplicates = df_footprint[duplicates]

print(df_duplicates)

In [ ]:
# Deleting all duplicates
df_footprint_nd = df_footprint.drop_duplicates(keep=False)

print(df_footprint_nd)

### 2.2.2 Drop Clients not in experiment

In [ ]:
#create list of client_ids: 
clients_list = list(df_clients['client_id'])
clients_list

Because we need to put an eye only in the clients who we know are in the experiment (so, the list of clients in df_clients), we are deleting the rest of clients. 

In [ ]:
#creating new dataframe with footprints only with clients from clients list
df_sample_fp = df_footprint_nd.loc[df_footprint_nd['client_id'].isin(clients_list)]

df_sample_fp.shape

### 2.2.3 Nulls

Let's see now if we have nulls values

In [ ]:
missing_data = df_footprint_nd.isnull().sum()
print(missing_data)

# Para comprobar si hay datos atípicos en la columna clnt_age, puedes calcular los límites intercuartílicos (IQR). 

In [77]:
# Calcular Q1 (primer cuartil) y Q3 (tercer cuartil)
Q1 = df_demo['clnt_age'].quantile(0.25)
Q3 = df_demo['clnt_age'].quantile(0.75)
IQR = Q3 - Q1

# Definir límites inferior y superior
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identificar valores atípicos
outliers = df_demo[(df_demo['clnt_age'] < lower_bound) | (df_demo['clnt_age'] > upper_bound)]

# Mostrar cantidad de valores atípicos
print(f'Cantidad de valores atípicos en clnt_age: {outliers.shape[0]}')


Cantidad de valores atípicos en clnt_age: 0


##  Análisis de la Demografía de los Clientes

In [78]:
import pandas as pd

# Cargar el DataFrame (si no está cargado)
# df_demo = pd.read_csv('ruta_al_archivo.csv')

# Análisis de la edad de los clientes
age_distribution = df_demo['clnt_age'].describe()

# Distribución de género
gender_distribution = df_demo['gendr'].value_counts()

# Número de cuentas por cliente
account_distribution = df_demo['num_accts'].value_counts()

age_distribution, gender_distribution, account_distribution


(count    70591.000000
 mean        46.442542
 std         15.591381
 min         13.500000
 25%         32.500000
 50%         47.000000
 75%         59.000000
 max         96.000000
 Name: clnt_age, dtype: float64,
 gendr
 U    24122
 M    23724
 F    22745
 Name: count, dtype: int64,
 num_accts
 2.0    55494
 3.0    12528
 4.0     2241
 5.0      284
 6.0       33
 7.0        8
 1.0        2
 8.0        1
 Name: count, dtype: int64)

In [79]:
# Edad promedio
average_age = df_demo['clnt_age'].mean()

# Tenencia promedio
average_tenure = df_demo['clnt_tenure_yr'].mean()

# Agrupar por edad y tenencia
age_tenure_analysis = df_demo.groupby(['clnt_tenure_yr', 'clnt_age']).size().reset_index(name='counts')

average_age, average_tenure, age_tenure_analysis


(46.442542250428524,
 12.053108753240497,
       clnt_tenure_yr  clnt_age  counts
 0                2.0      14.5       1
 1                2.0      17.0       1
 2                2.0      18.5       2
 3                2.0      19.5       1
 4                2.0      21.5       1
 ...              ...       ...     ...
 4345            55.0      67.0       1
 4346            55.0      68.0       1
 4347            55.0      72.0       1
 4348            55.0      78.0       1
 4349            62.0      51.0       1
 
 [4350 rows x 3 columns])

In [80]:
# Análisis de saldo en función de la edad
age_balance_correlation = df_demo[['clnt_age', 'bal']].corr()

# Promedio de llamadas y logins por grupo de saldo
call_login_analysis = df_demo.groupby(pd.cut(df_demo['bal'], bins=[0, 100000, 500000, 1000000, 10000000])).agg({'calls_6_mnth': 'mean', 'logons_6_mnth': 'mean'})

age_balance_correlation, call_login_analysis


C:\Users\Usuario\AppData\Local\Temp\ipykernel_120500\287947343.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  call_login_analysis = df_demo.groupby(pd.cut(df_demo['bal'], bins=[0, 100000, 500000, 1000000, 10000000])).agg({'calls_6_mnth': 'mean', 'logons_6_mnth': 'mean'})


(          clnt_age       bal
 clnt_age  1.000000  0.209545
 bal       0.209545  1.000000,
                      calls_6_mnth  logons_6_mnth
 bal                                             
 (0, 100000]              3.053574       5.207998
 (100000, 500000]         3.902361       6.150634
 (500000, 1000000]        4.567957       6.771071
 (1000000, 10000000]      4.882653       7.113946)

In [81]:
correlation_age_accounts = df_demo['clnt_age'].corr(df_demo['num_accts'])
print("Correlación entre Edad y Número de Cuentas:", correlation_age_accounts)


Correlación entre Edad y Número de Cuentas: -0.017708400750834206


In [82]:
age_bins = [0, 30, 45, 60, 100]
age_labels = ['Joven', 'Adulto', 'Mayor', 'Anciano']
df_demo['age_group'] = pd.cut(df_demo['clnt_age'], bins=age_bins, labels=age_labels)

avg_accounts_by_age_group = df_demo.groupby('age_group')['num_accts'].mean()
print(avg_accounts_by_age_group)


age_group
Joven      2.264922
Adulto     2.263316
Mayor      2.254641
Anciano    2.238759
Name: num_accts, dtype: float64


C:\Users\Usuario\AppData\Local\Temp\ipykernel_120500\218544797.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  avg_accounts_by_age_group = df_demo.groupby('age_group')['num_accts'].mean()


In [83]:
demographics_balance = df_demo.groupby(['clnt_age', 'gendr'])['bal'].describe()
demographics_balance


count       mean            std        min          25%  \
clnt_age gendr                                                            
13.5     U        1.0   24435.08            NaN   24435.08   24435.0800   
14.0     F        1.0   16989.14            NaN   16989.14   16989.1400   
         U        1.0   19945.35            NaN   19945.35   19945.3500   
14.5     F        3.0   17103.58    1781.543845   15048.83   16546.8800   
         M        4.0   23983.90    7106.456638   15487.91   19442.9675   
...               ...        ...            ...        ...          ...   
94.0     U        3.0  394455.75  485296.364159   58162.77  116286.6300   
94.5     U        1.0  108825.59            NaN  108825.59  108825.5900   
95.5     F        1.0   43471.96            NaN   43471.96   43471.9600   
96.0     M        1.0   36297.66            NaN   36297.66   36297.6600   
         U        1.0   94240.29            NaN   94240.29   94240.2900   

                       50%          75%        max  
clnt_age gendr                                      
13.5     U       24435.080   24435.0800   24435.08  
14.0     F       16989.140   16989.1400   16989.14  
         U       19945.350   19945.3500   19945.35  
14.5     F       18044.930   18130.9550   18216.98  
         M       25116.555   29657.4875   30214.58  
...                    ...          ...        ...  
94.0     U      174410.490  562602.2400  950793.99  
94.5     U      108825.590  108825.5900  108825.59  
95.5     F       43471.960   43471.9600   43471.96  
96.0     M       36297.660   36297.6600   36297.66  
         U       94240.290   94240.2900   94240.29  

[481 rows x 8 columns]